In [569]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import warnings

from retrosheet import get_batting_results as gbr
from retrosheet import score_value_matrix as svm

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [8]:
#イベントデータ
# data2016 = pd.read_csv('C:/Users/nukan/Desktop/work/25_Python/MLB_analysis/data/play_by_play_2016.csv', sep='\t')
df = pd.read_csv('data/play_by_play_2001-2019.csv', sep='\t')
print(df.shape)
df.head(30)



(3650188, 34)


,game_id,inn_ct,bat_home_id,inn_pa_ct,outs_ct,balls_ct,strikes_ct,bat_lineup_id,event_tx,away_score_ct,home_score_ct,bat_id,resp_bat_hand_cd,pit_id,start_bases_cd,event_cd,bat_event_fl,ab_fl,sf_fl,bunt_fl,h_cd,bat_dest_id,base1_run_id,base2_run_id,base3_run_id,run1_dest_id,run2_dest_id,run3_dest_id,event_outs_ct,battedball_cd,battedball_loc_tx,fld_cd,rbi_ct,pitch_seq_tx
0,OAK201406130,6,1,0,0,3,2,1,W,3,0,crisc001,L,pheld001,0,14,T,F,F,F,0,1,NaN,NaN,NaN,0,0,0,0,NaN,NaN,0,0,CBBCBFB
1,OAK201406130,6,1,1,0,3,2,2,36(1)1/GDP,3,0,jasoj001,L,pheld001,1,2,T,T,F,F,0,0,crisc001,NaN,NaN,0,0,0,2,G,NaN,3,0,B1CFBBX
2,OAK201406130,6,1,2,2,1,0,3,63/G,3,0,donaj001,R,pheld001,0,2,T,T,F,F,0,0,NaN,NaN,NaN,0,0,0,1,G,NaN,6,0,BX
3,OAK201406130,7,0,0,0,0,2,9,K,3,0,johnk003,L,abadf001,0,3,T,T,F,F,0,0,NaN,NaN,NaN,0,0,0,1,NaN,NaN,0,0,.CFT
4,OAK201406130,7,0,1,1,2,1,1,HP,3,0,gardb001,L,abadf001,0,16,T,F,F,F,0,1,NaN,NaN,NaN,0,0,0,0,NaN,NaN,0,0,BBCH
5,OAK201406130,7,0,2,1,0,1,2,46(1)3/GDP,3,0,jeted001,R,abadf001,1,2,T,T,F,F,0,0,gardb001,NaN,NaN,0,0,0,2,G,NaN,4,0,C1X
6,OAK201406130,7,1,0,0,2,0,4,7/F7LF,3,0,mossb001,L,pheld001,0,2,T,T,F,F,0,0,NaN,NaN,NaN,0,0,0,1,F,7LF,7,0,BBX
7,OAK201406130,7,1,1,1,1,2,5,K,3,0,cespy001,R,pheld001,0,3,T,T,F,F,0,0,NaN,NaN,NaN,0,0,0,1,NaN,NaN,0,0,CBFC
8,OAK201406130,7,1,2,2,2,1,6,D7/L,3,0,lowrj001,L,pheld001,0,21,T,T,F,F,2,2,NaN,NaN,NaN,0,0,0,0,L,NaN,7,0,BSBX
9,OAK201406130,7,1,3,2,0,1,7,3/P3F,3,0,norrd001,R,betad001,2,2,T,T,F,F,0,0,NaN,lowrj001,NaN,0,2,0,1,P,3F,3,0,.CX


In [10]:
#ゲーム年
df['game_dt'] = pd.to_datetime(df['game_id'].str[3:11])
df['game_year'] = df['game_dt'].dt.strftime('%Y')
df.head()

,game_id,inn_ct,bat_home_id,inn_pa_ct,outs_ct,balls_ct,strikes_ct,bat_lineup_id,event_tx,away_score_ct,home_score_ct,bat_id,resp_bat_hand_cd,pit_id,start_bases_cd,event_cd,bat_event_fl,ab_fl,sf_fl,bunt_fl,h_cd,bat_dest_id,base1_run_id,base2_run_id,base3_run_id,run1_dest_id,run2_dest_id,run3_dest_id,event_outs_ct,battedball_cd,battedball_loc_tx,fld_cd,rbi_ct,pitch_seq_tx,game_dt,game_year
0,OAK201406130,6,1,0,0,3,2,1,W,3,0,crisc001,L,pheld001,0,14,T,F,F,F,0,1,NaN,NaN,NaN,0,0,0,0,NaN,NaN,0,0,CBBCBFB,2014-06-13,2014
1,OAK201406130,6,1,1,0,3,2,2,36(1)1/GDP,3,0,jasoj001,L,pheld001,1,2,T,T,F,F,0,0,crisc001,NaN,NaN,0,0,0,2,G,NaN,3,0,B1CFBBX,2014-06-13,2014
2,OAK201406130,6,1,2,2,1,0,3,63/G,3,0,donaj001,R,pheld001,0,2,T,T,F,F,0,0,NaN,NaN,NaN,0,0,0,1,G,NaN,6,0,BX,2014-06-13,2014
3,OAK201406130,7,0,0,0,0,2,9,K,3,0,johnk003,L,abadf001,0,3,T,T,F,F,0,0,NaN,NaN,NaN,0,0,0,1,NaN,NaN,0,0,.CFT,2014-06-13,2014
4,OAK201406130,7,0,1,1,2,1,1,HP,3,0,gardb001,L,abadf001,0,16,T,F,F,F,0,1,NaN,NaN,NaN,0,0,0,0,NaN,NaN,0,0,BBCH,2014-06-13,2014


In [40]:
#2016年のゲーム
data2016 = df[df['game_year']=='2016']
print(data2016.shape)
data2016.head()

#バッティングベントのみ
data2016 = data2016[data2016['bat_event_fl']=='T']
print(data2016.shape)
data2016.head()

(190715, 36)
(184580, 36)


,game_id,inn_ct,bat_home_id,inn_pa_ct,outs_ct,balls_ct,strikes_ct,bat_lineup_id,event_tx,away_score_ct,home_score_ct,bat_id,resp_bat_hand_cd,pit_id,start_bases_cd,event_cd,bat_event_fl,ab_fl,sf_fl,bunt_fl,h_cd,bat_dest_id,base1_run_id,base2_run_id,base3_run_id,run1_dest_id,run2_dest_id,run3_dest_id,event_outs_ct,battedball_cd,battedball_loc_tx,fld_cd,rbi_ct,pitch_seq_tx,game_dt,game_year
830179,ANA201604040,1,0,0,0,1,0,1,D9/G+,0,0,fowld001,L,richg002,0,21,T,T,F,F,2,2,NaN,NaN,NaN,0,0,0,0,G,NaN,9,0,BX,2016-04-04,2016
830180,ANA201604040,1,0,1,0,0,0,2,63/G.2-3,0,0,heywj001,L,richg002,2,2,T,T,F,F,0,0,NaN,fowld001,NaN,0,3,0,1,G,NaN,6,0,X,2016-04-04,2016
830181,ANA201604040,1,0,2,1,0,2,3,K,0,0,zobrb001,L,richg002,4,3,T,T,F,F,0,0,NaN,NaN,fowld001,0,0,3,1,NaN,NaN,0,0,SFS,2016-04-04,2016
830182,ANA201604040,1,0,3,2,1,1,4,S8/G.3-H,0,0,rizza001,L,richg002,4,20,T,T,F,F,1,1,NaN,NaN,fowld001,0,0,4,0,G,NaN,8,1,BCX,2016-04-04,2016
830183,ANA201604040,1,0,4,2,2,2,5,K23,1,0,bryak001,R,richg002,1,3,T,T,F,F,0,0,rizza001,NaN,NaN,1,0,0,1,NaN,NaN,0,0,BSS*B1S,2016-04-04,2016


In [42]:
#状態の取得
data2016 = svm(data2016)
print(data2016.shape)
data2016.head(30)

(184580, 39)


,game_id,inn_ct,bat_home_id,inn_pa_ct,outs_ct,balls_ct,strikes_ct,bat_lineup_id,event_tx,away_score_ct,home_score_ct,bat_id,resp_bat_hand_cd,pit_id,start_bases_cd,event_cd,bat_event_fl,ab_fl,sf_fl,bunt_fl,h_cd,bat_dest_id,base1_run_id,base2_run_id,base3_run_id,run1_dest_id,run2_dest_id,run3_dest_id,event_outs_ct,battedball_cd,battedball_loc_tx,fld_cd,rbi_ct,pitch_seq_tx,game_dt,game_year,state,new.state,run_value
0,ANA201604040,1,0,0,0,1,0,1,D9/G+,0,0,fowld001,L,richg002,0,21,T,T,F,F,2,2,nan,nan,nan,0,0,0,0,G,NaN,9,0,BX,2016-04-04,2016,0000,0100,0.610775
1,ANA201604040,1,0,1,0,0,0,2,63/G.2-3,0,0,heywj001,L,richg002,2,2,T,T,F,F,0,0,nan,fowld001,nan,0,3,0,1,G,NaN,6,0,X,2016-04-04,2016,0100,0011,-0.211902
2,ANA201604040,1,0,2,1,0,2,3,K,0,0,zobrb001,L,richg002,4,3,T,T,F,F,0,0,nan,nan,fowld001,0,0,3,1,NaN,NaN,0,0,SFS,2016-04-04,2016,0011,0012,-0.562117
3,ANA201604040,1,0,3,2,1,1,4,S8/G.3-H,0,0,rizza001,L,richg002,4,20,T,T,F,F,1,1,nan,nan,fowld001,0,0,4,0,G,NaN,8,1,BCX,2016-04-04,2016,0012,1002,0.883453
4,ANA201604040,1,0,4,2,2,2,5,K23,1,0,bryak001,R,richg002,1,3,T,T,F,F,0,0,rizza001,nan,nan,1,0,0,1,NaN,NaN,0,0,BSS*B1S,2016-04-04,2016,1002,1003,-0.211358
5,ANA201604040,1,1,0,0,2,0,1,13/G-,1,0,escoy001,R,arrij001,0,2,T,T,F,F,0,0,nan,nan,nan,0,0,0,1,G,NaN,1,0,BBX,2016-04-04,2016,0000,0001,-0.227359
6,ANA201604040,1,1,1,1,1,1,2,8/F,1,0,navad002,L,arrij001,0,2,T,T,F,F,0,0,nan,nan,nan,0,0,0,1,F,NaN,8,0,BCX,2016-04-04,2016,0001,0002,-0.163178
7,ANA201604040,1,1,2,2,0,1,3,4/L,1,0,troum001,R,arrij001,0,2,T,T,F,F,0,0,nan,nan,nan,0,0,0,1,L,NaN,4,0,CX,2016-04-04,2016,0002,0003,-0.100613
8,ANA201604040,2,0,0,0,1,2,6,K,1,0,schwk001,L,richg002,0,3,T,T,F,F,0,0,nan,nan,nan,0,0,0,1,NaN,NaN,0,0,BCCS,2016-04-04,2016,0000,0001,-0.227359
9,ANA201604040,2,0,1,1,1,2,7,13/G-,1,0,solej001,R,richg002,0,2,T,T,F,F,0,0,nan,nan,nan,0,0,0,1,G,NaN,1,0,SBFX,2016-04-04,2016,0001,0002,-0.163178


In [43]:
#3アウトの状態は一律「３」とする
import re
data2016['new.state'] = data2016['new.state'].apply(lambda x: re.sub("[0-1]{3}3", '3', x))
data2016.head(30)

,game_id,inn_ct,bat_home_id,inn_pa_ct,outs_ct,balls_ct,strikes_ct,bat_lineup_id,event_tx,away_score_ct,home_score_ct,bat_id,resp_bat_hand_cd,pit_id,start_bases_cd,event_cd,bat_event_fl,ab_fl,sf_fl,bunt_fl,h_cd,bat_dest_id,base1_run_id,base2_run_id,base3_run_id,run1_dest_id,run2_dest_id,run3_dest_id,event_outs_ct,battedball_cd,battedball_loc_tx,fld_cd,rbi_ct,pitch_seq_tx,game_dt,game_year,state,new.state,run_value
0,ANA201604040,1,0,0,0,1,0,1,D9/G+,0,0,fowld001,L,richg002,0,21,T,T,F,F,2,2,nan,nan,nan,0,0,0,0,G,NaN,9,0,BX,2016-04-04,2016,0000,0100,0.610775
1,ANA201604040,1,0,1,0,0,0,2,63/G.2-3,0,0,heywj001,L,richg002,2,2,T,T,F,F,0,0,nan,fowld001,nan,0,3,0,1,G,NaN,6,0,X,2016-04-04,2016,0100,0011,-0.211902
2,ANA201604040,1,0,2,1,0,2,3,K,0,0,zobrb001,L,richg002,4,3,T,T,F,F,0,0,nan,nan,fowld001,0,0,3,1,NaN,NaN,0,0,SFS,2016-04-04,2016,0011,0012,-0.562117
3,ANA201604040,1,0,3,2,1,1,4,S8/G.3-H,0,0,rizza001,L,richg002,4,20,T,T,F,F,1,1,nan,nan,fowld001,0,0,4,0,G,NaN,8,1,BCX,2016-04-04,2016,0012,1002,0.883453
4,ANA201604040,1,0,4,2,2,2,5,K23,1,0,bryak001,R,richg002,1,3,T,T,F,F,0,0,rizza001,nan,nan,1,0,0,1,NaN,NaN,0,0,BSS*B1S,2016-04-04,2016,1002,3,-0.211358
5,ANA201604040,1,1,0,0,2,0,1,13/G-,1,0,escoy001,R,arrij001,0,2,T,T,F,F,0,0,nan,nan,nan,0,0,0,1,G,NaN,1,0,BBX,2016-04-04,2016,0000,0001,-0.227359
6,ANA201604040,1,1,1,1,1,1,2,8/F,1,0,navad002,L,arrij001,0,2,T,T,F,F,0,0,nan,nan,nan,0,0,0,1,F,NaN,8,0,BCX,2016-04-04,2016,0001,0002,-0.163178
7,ANA201604040,1,1,2,2,0,1,3,4/L,1,0,troum001,R,arrij001,0,2,T,T,F,F,0,0,nan,nan,nan,0,0,0,1,L,NaN,4,0,CX,2016-04-04,2016,0002,3,-0.100613
8,ANA201604040,2,0,0,0,1,2,6,K,1,0,schwk001,L,richg002,0,3,T,T,F,F,0,0,nan,nan,nan,0,0,0,1,NaN,NaN,0,0,BCCS,2016-04-04,2016,0000,0001,-0.227359
9,ANA201604040,2,0,1,1,1,2,7,13/G-,1,0,solej001,R,richg002,0,2,T,T,F,F,0,0,nan,nan,nan,0,0,0,1,G,NaN,1,0,SBFX,2016-04-04,2016,0001,0002,-0.163178


In [56]:
# import csv
# data2016[['bat_id', 'state', 'new.state', 'run_value']].to_csv('data/data2016.csv', sep='\t', quoting=csv.QUOTE_NONNUMERIC, index=None)

In [423]:
P_matrix = data2016.groupby(['state', 'new.state'], as_index=False).agg({'bat_id': 'count'})
len_state = data2016.groupby('state', as_index=False).agg({'bat_id': 'count'})
P_matrix = pd.merge(P_matrix, len_state, on='state')
P_matrix['prob'] = P_matrix['bat_id_x'] / P_matrix['bat_id_y']
P_matrix = pd.pivot_table(P_matrix, values='prob', index='state', columns='new.state')
P_matrix.fillna(0, inplace=True)

P_matrix.loc['3'] = 1
P_matrix

new.state,0000,0001,0002,0010,0011,0012,0100,0101,0102,0110,0111,0112,1000,1001,1002,1010,1011,1012,1100,1101,1102,1110,1111,1112,3
state,,,,,,,,,,,,,,,,,,,,,,,,,
0000,0.033703,0.674613,0.000000,0.005646,0.000000,0.000000,0.050423,0.000000,0.000000,0.000000,0.000000,0.000000,0.235615,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0001,0.000000,0.029070,0.685237,0.000000,0.005164,0.000000,0.000000,0.046476,0.000000,0.000000,0.000000,0.000000,0.000000,0.234052,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0002,0.000000,0.000000,0.031972,0.000000,0.000000,0.004616,0.000000,0.000000,0.045329,0.000000,0.000000,0.000000,0.000000,0.000000,0.235423,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.682659
0010,0.023346,0.214008,0.005837,0.009728,0.437743,0.000000,0.048638,0.001946,0.000000,0.000000,0.000000,0.000000,0.140078,0.007782,0.000000,0.110895,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0011,0.000000,0.021358,0.204819,0.000000,0.006024,0.353779,0.000000,0.046550,0.005476,0.000000,0.000000,0.000000,0.000000,0.182913,0.019168,0.000000,0.152245,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007667
0012,0.000000,0.000000,0.021927,0.000000,0.000000,0.006111,0.000000,0.000000,0.041697,0.000000,0.000000,0.000000,0.000000,0.000000,0.129763,0.000000,0.000000,0.162114,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.638390
0100,0.030091,0.003951,0.005775,0.005167,0.247720,0.000000,0.053191,0.386930,0.000000,0.004559,0.000000,0.000000,0.052280,0.015805,0.000000,0.093009,0.000000,0.000000,0.101520,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0101,0.000000,0.029452,0.004291,0.000000,0.005461,0.187439,0.000000,0.047006,0.443534,0.000000,0.002926,0.000000,0.000000,0.061830,0.010728,0.000000,0.065925,0.000000,0.000000,0.135167,0.000000,0.000000,0.000000,0.000000,0.006241
0102,0.000000,0.000000,0.023845,0.000000,0.000000,0.005849,0.000000,0.000000,0.058188,0.000000,0.000000,0.000450,0.000000,0.000000,0.075135,0.000000,0.000000,0.032543,0.000000,0.000000,0.155969,0.000000,0.000000,0.000000,0.648020


In [427]:
tmp_list = []
for i in range(0, len(P_matrix.index)-1):
    tmp_row = 0
    for k in range(0,4):
        tmp = int(P_matrix.index[i][k])
        tmp_row = tmp_row + tmp
#     print(tmp_row)
    for j in range(0, len(P_matrix.columns)-1):
        tmp_col = 0
        for k in range(0,4):
            tmp = int(P_matrix.columns[j][k])
            tmp_col = tmp_col + tmp
#         print(tmp_col)
        tmp_score = 1 + tmp_row - tmp_col
        tmp_list.append([P_matrix.index[i], P_matrix.columns[j], tmp_score])
        
R = pd.DataFrame(data=tmp_list)
R.columns = ['state', 'new.state', 'run']

R = pd.pivot_table(R, values='run', index='state', columns='new.state')
R['V25'] = 0
print(R.shape)
R

(24, 25)


new.state,0000,0001,0002,0010,0011,0012,0100,0101,0102,0110,0111,0112,1000,1001,1002,1010,1011,1012,1100,1101,1102,1110,1111,1112,V25
state,,,,,,,,,,,,,,,,,,,,,,,,,
0000,1,0,-1,0,-1,-2,0,-1,-2,-1,-2,-3,0,-1,-2,-1,-2,-3,-1,-2,-3,-2,-3,-4,0
0001,2,1,0,1,0,-1,1,0,-1,0,-1,-2,1,0,-1,0,-1,-2,0,-1,-2,-1,-2,-3,0
0002,3,2,1,2,1,0,2,1,0,1,0,-1,2,1,0,1,0,-1,1,0,-1,0,-1,-2,0
0010,2,1,0,1,0,-1,1,0,-1,0,-1,-2,1,0,-1,0,-1,-2,0,-1,-2,-1,-2,-3,0
0011,3,2,1,2,1,0,2,1,0,1,0,-1,2,1,0,1,0,-1,1,0,-1,0,-1,-2,0
0012,4,3,2,3,2,1,3,2,1,2,1,0,3,2,1,2,1,0,2,1,0,1,0,-1,0
0100,2,1,0,1,0,-1,1,0,-1,0,-1,-2,1,0,-1,0,-1,-2,0,-1,-2,-1,-2,-3,0
0101,3,2,1,2,1,0,2,1,0,1,0,-1,2,1,0,1,0,-1,1,0,-1,0,-1,-2,0
0102,4,3,2,3,2,1,3,2,1,2,1,0,3,2,1,2,1,0,2,1,0,1,0,-1,0


In [483]:
import random

def simulate_half_inning(P_matrix, R, start=0):

    s = start
    runs = 0

    while s < 24:
        s_new = random.choices(np.arange(0 ,25, 1), k=1, weights=P_matrix.iloc[s].values)
        s_new = s_new[0]
#         print('s_new: {}'.format(s_new))
        runs = runs + R.iloc[s, s_new]
        s = s_new

    return runs

In [571]:
simulation_list = []
for i in range(0, 10000):
    simulation_list.append(simulate_half_inning(P_matrix, R))

simulation_list = pd.DataFrame(simulation_list)
simulation_list.columns = ['cnt']
simulation_list

,cnt
0,0
1,0
2,0
3,0
4,0
...,...
9995,0
9996,0
9997,5
9998,1


In [574]:
simulation_list['cnt'].value_counts() / simulation_list['cnt'].value_counts().sum()

0     0.7336
1     0.1471
2     0.0674
3     0.0289
4     0.0125
5     0.0062
6     0.0021
7     0.0012
8     0.0004
9     0.0002
10    0.0002
11    0.0002
Name: cnt, dtype: float64

In [577]:
simulation_list = []
for i in range(0, 10000):
    simulation_list.append(simulate_half_inning(P_matrix, R))
np.mean(simulation_list)

0.4939